<a href="https://colab.research.google.com/github/lucas-azdias/Scripts-Universidade/blob/main/Python/Big%20Data/13/PairRDDs/PairRDDs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Instalando PySpark
!pip install PySpark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for PySpark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=f19a66d1d7ee6268079471beec7fc38c5e7749cd2b436eaf50d1d0f471d6d0af
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built PySpark


In [2]:
# Importando as bibliotecas
import pyspark  # Biblioteca principal
from pyspark.sql import SparkSession  # Biblioteca para a criação de uma sessão Spark

In [3]:
# Criação de uma sessão Spark
spark = SparkSession.builder.appName("Airports").master('local[*]').getOrCreate()

In [4]:
# Criação do SparkContext (conexão entre o Driver e Workers)
sc = spark.sparkContext

In [35]:
rdd_airports = sc.textFile("Airports.csv")
rdd_realestate = sc.textFile("RealEstate.csv")

# PairRDDs 1

In [7]:
# PairRDDs 1
rdd_airports_no_header = rdd_airports.filter(lambda x: not "name" in x) # Filtra header
rdd_pair_airport_city_country = rdd_airports_no_header.map(lambda x: (x.split(",")[2], (x.split(",")[3], x.split(",")[4])))

[('GOROKA', ('GOROKA', 'PAPUA NEW GUINEA')),
 ('N/A', ('LAE', 'PAPUA NEW GUINEA')),
 ('MADANG', ('MADANG', 'PAPUA NEW GUINEA')),
 ('MOUNT HAGEN', ('MOUNT HAGEN', 'PAPUA NEW GUINEA')),
 ('NADZAB', ('NADZAB', 'PAPUA NEW GUINEA'))]

In [9]:
# Aeroportos em Paris
rdd_pair_paris = rdd_pair_airport_city_country.filter(lambda x: x[1][0].upper() == "PARIS")
rdd_pair_paris.take(5)

[('LE BOURGET', ('PARIS', 'FRANCE')),
 ('CHARLES DE GAULLE', ('PARIS', 'FRANCE')),
 ('N/A', ('PARIS', 'FRANCE')),
 ('ORLY', ('PARIS', 'FRANCE'))]

In [11]:
# Aeroportos em Paris capitalizado
rdd_pair_paris_cap = rdd_pair_paris.mapValues(lambda x: (x[0].capitalize(), x[1].capitalize()))
rdd_pair_paris_cap.take(5)

[('LE BOURGET', ('Paris', 'France')),
 ('CHARLES DE GAULLE', ('Paris', 'France')),
 ('N/A', ('Paris', 'France')),
 ('ORLY', ('Paris', 'France'))]

In [12]:
# Aeroportos em Paris capitalizado ordenado pela chave
rdd_pair_paris_sort = rdd_pair_paris_cap.sortByKey(ascending=True)
rdd_pair_paris_sort.take(5)

[('CHARLES DE GAULLE', ('Paris', 'France')),
 ('LE BOURGET', ('Paris', 'France')),
 ('N/A', ('Paris', 'France')),
 ('ORLY', ('Paris', 'France'))]

In [13]:
rdd_pair_paris_sort.saveAsTextFile("rdd_pair_paris_sort.txt")

# PairRDDs 2

In [16]:
# PairRDDs 2
rdd_airports_no_header = rdd_airports.filter(lambda x: not "name" in x) # Filtra header
rdd_airports_pair = rdd_airports_no_header.map(lambda x: (x.split(",")[2], (x.split(",")[3], int(x.split(",")[13]))))
rdd_airports_pair.take(5)

[('GOROKA', ('GOROKA', 1610)),
 ('N/A', ('LAE', 0)),
 ('MADANG', ('MADANG', 7)),
 ('MOUNT HAGEN', ('MOUNT HAGEN', 1643)),
 ('NADZAB', ('NADZAB', 73))]

In [17]:
rdd_airports_filtered = rdd_airports_pair.filter(lambda x: x[1][0].upper() == "PARIS" and x[1][1] > 80)
rdd_airports_filtered.take(5)

[('CHARLES DE GAULLE', ('PARIS', 120)), ('ORLY', ('PARIS', 89))]

In [20]:
rdd_airports_cap = rdd_airports_filtered.mapValues(lambda x: (x[0].capitalize(), x[1]))
rdd_airports_cap.take(5)

[('CHARLES DE GAULLE', ('Paris', 120)), ('ORLY', ('Paris', 89))]

In [21]:
rdd_airports_sorted = rdd_airports_cap.sortByKey()
rdd_airports_sorted.take(5)

[('CHARLES DE GAULLE', ('Paris', 120)), ('ORLY', ('Paris', 89))]

In [22]:
rdd_airports_sorted.saveAsTextFile("rdd_airports_sorted.txt")

# PairRDDs 3

In [24]:
# PairRDDs 3
rdd_airports_no_header = rdd_airports.filter(lambda x: not "name" in x) # Filtra header
rdd_pair_country_occurs = rdd_airports_no_header.map(lambda x: (x.split(",")[4].lower(), 1))
rdd_pair_country_occurs.take(5)

[('papua new guinea', 1),
 ('papua new guinea', 1),
 ('papua new guinea', 1),
 ('papua new guinea', 1),
 ('papua new guinea', 1)]

In [26]:
rdd_pair_country_count = rdd_pair_country_occurs.reduceByKey(lambda x, y: x + y)
rdd_pair_country_count.take(5)

[('new zealand', 214),
 ('canada', 259),
 ('algeria', 49),
 ('afghanistan', 94),
 ('benin', 9)]

In [33]:
rdd_pair_country_sorted = rdd_pair_country_count.sortBy(lambda x: x[1], ascending=False)
rdd_pair_country_sorted.take(20)

[('usa', 552),
 ('germany', 529),
 ('france', 472),
 ('south africa', 262),
 ('canada', 259),
 ('iran', 256),
 ('zaire', 249),
 ('indonesia', 218),
 ('china', 218),
 ('new zealand', 214),
 ('russia', 189),
 ('india', 186),
 ('bolivia', 180),
 ('peru', 171),
 ('italy', 165),
 ('sweden', 161),
 ('england', 146),
 ('ecuador', 146),
 ('brazil', 142),
 ('argentina', 142)]

# PairRDDs 4

In [38]:
rdd_realestate_no_header = rdd_realestate.filter(lambda x: x.split(",")[0] != "MLS")
rdd_realestate_pair = rdd_realestate_no_header.map(lambda x: (int(x.split(",")[3]), (float(x.split(",")[2]), 1)))
rdd_realestate_pair.take(5)

[(3, (795000.0, 1)),
 (4, (399000.0, 1)),
 (4, (545000.0, 1)),
 (4, (909000.0, 1)),
 (3, (109900.0, 1))]

In [40]:
rdd_realestate_tot_price_per_room = rdd_realestate_pair.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
rdd_realestate_tot_price_per_room.take(5)

[(4, (85575190.0, 177)),
 (2, (32761834.0, 123)),
 (0, (586900.0, 2)),
 (6, (2412900.0, 4)),
 (10, (699000.0, 1))]

In [42]:
rdd_realestate_avg_price_per_room = rdd_realestate_tot_price_per_room.map(lambda x: (x[0], x[1][0] / x[1][1]))
rdd_realestate_avg_price_per_room.take(5)

[(4, 483475.6497175141),
 (2, 266356.3739837398),
 (0, 293450.0),
 (6, 603225.0),
 (10, 699000.0)]

In [45]:
rdd_realestate_avg_price_per_room_sorted = rdd_realestate_avg_price_per_room.sortByKey(ascending=False)
rdd_realestate_avg_price_per_room_sorted.collect()

[(10, 699000.0),
 (7, 325000.0),
 (6, 603225.0),
 (5, 657858.0645161291),
 (4, 483475.6497175141),
 (3, 359062.20649651974),
 (2, 266356.3739837398),
 (1, 169981.81818181818),
 (0, 293450.0)]